In [25]:
import pandas as pd
import numpy as np
import random

from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
from lambdaMART import LambdaMART

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
def dcg(rel, k=None):
    i = np.arange(1, len(rel)+1)
    gain = (2**rel - 1)/np.log2(i + 1)
    if k is not None:
        gain = gain[i <= k]
    return gain.sum()

In [28]:
def idcg(rel, k=None):
    rel = np.sort(rel)[::-1]
    i = np.arange(1, len(rel)+1)
    gain = (2**rel - 1)/np.log2(i + 1)
    if k is not None:
        gain = gain[i <= k]
    return gain.sum()

In [29]:
def ndcg(rel, k=None):
    idcg_value = idcg(rel, k=k)
    if idcg_value != 0:
        return dcg(rel, k=k) / idcg_value
    else:
        return 0

In [30]:
def ndcg_mean(res_table, k=None):
    ndcg_val = 0
    for qid in res_table['QueryId'].unique():
        rel = res_table[res_table['QueryId'] == qid]['rel']
        ndcg_val += ndcg(rel, k=k)
    return ndcg_val / res_table['QueryId'].nunique()

In [31]:
df, rank, qid = load_svmlight_file('data/train.txt', query_id = True)

In [32]:
df_test, rank_test, qid_test = load_svmlight_file('data/test.txt', query_id = True)

In [33]:
print(f'Number of unique queries in the dataset: {len(np.unique(qid))}')

Number of unique queries in the dataset: 471


In [53]:
sample_size = 50
sample_queries = random.sample(list(np.unique(qid)), sample_size)
qid_from_sample = (qid == sample_queries[0])
for idx in sample_queries[1:]:
    qid_from_sample |= (qid == idx)
df_part = df[qid_from_sample]
rank_part = rank[qid_from_sample]
qid_part = qid[qid_from_sample]

In [54]:
train_idx, cv_idx = train_test_split(np.unique(qid_part), test_size=0.2)

In [55]:
qid_has_train_idx = (qid_part == train_idx[0])
for idx in train_idx[1:]:
    qid_has_train_idx |= (qid_part == idx)

In [56]:
qid_has_cv_idx = (qid_part == cv_idx[0])
for idx in cv_idx[1:]:
    qid_has_cv_idx |= (qid_part == idx)

In [57]:
df_train = df_part[qid_has_train_idx]
rank_train = rank_part[qid_has_train_idx]
qid_train = qid_part[qid_has_train_idx]

df_cv = df_part[qid_has_cv_idx]
rank_cv = rank_part[qid_has_cv_idx]
qid_cv = qid_part[qid_has_cv_idx]

In [58]:
len(rank_train)

1064

In [59]:
model = LambdaMART(num_trees=100, max_depth=4, learning_rate=0.125)
model.fit(df_train, rank_train, qid_train)

In [60]:
cv_predictions = model.predict(df_cv)

In [61]:
cv_res = pd.DataFrame({'neg_pred': -cv_predictions, 'QueryId': qid_cv, 
                       'DocumentId': np.arange(1, len(qid_cv)+1), 'rel': rank_cv})
cv_res = cv_res.sort_values(by=['QueryId', 'neg_pred'])

In [62]:
ndcg_mean(cv_res, k=5)

0.3657082528509856

In [63]:
predictions = model.predict(df_test)

In [64]:
res = pd.DataFrame({'neg_pred': -predictions, 'QueryId': qid_test, 'DocumentId': np.arange(1, len(qid_test)+1)})

In [65]:
res = res.sort_values(by=['QueryId', 'neg_pred'])

In [66]:
res[['QueryId', 'DocumentId']].to_csv('ranking_result.csv', index=False)